In [2]:
from langchain_ollama.llms import OllamaLLM
from langchain.prompts import ChatPromptTemplate

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('D:\dispatching-requests\data\clear_data.csv')
df

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\HONOR\AppData\Local\Temp\ipykernel_9932\2909424392.py:1: SyntaxWarning: invalid escape sequence '\d'
  df = pd.read_csv('D:\dispatching-requests\data\clear_data.csv')


,Тема,Описание,Тип оборудования,Точка отказа,Серийный номер
0,+BB0EPgRDBEIEMQRDBDo- +BCEEGAQbBBA- +BB0EGg-2-...,Здравствуйте.\r\n\r\n Сломался ...,Ноутбук,Блок питания,C222090381
1,/ шумит кулер / отклеились ножки,"Коллеги, добрый день.\r\n\r\nПрошу осуществить...",Ноутбук,Вентилятор,C223010310
2,10552. СИЛА HK-1404 // замена блоков питания //,"Добрый день! Не работают блоки питания, прошу ...",Ноутбук,Блок питания,"C223010423, C223011174"
3,10979. Ноутбук СИЛА НК2-1404 // не включается,Добрый день! Ноутбук не включается,Ноутбук,Материнская плата,C223012335
4,BIOS'ы,"Здравствуйте!\r\nПоделитесь, пожалуйста, после...",Ноутбук,Консультация,C223010345
...,...,...,...,...,...
180,по неисправной плашке памяти на dd203 - Enclos...,Добрый день!\r\nПросьба открыть заявку по неис...,СХД,Оперативная память,CKM00193800300
181,полосы на матрице НК2-3404,Полосы на матрице.,Ноутбук,Матрица,C223090893
182,полосы на экране и отклеились ножки,Переодически (было 3-4 раза) появляются полосы...,Ноутбук,Матрица,c223012503
183,ремонт ноутбука СИЛА НК2-1404,не работает порт usd и порт type-c на ноутбуке...,Ноутбук,Материнская плата,C223100081


In [10]:
title = df['Тема'][0]
body = df['Описание'][0]
type_of_equipment = df['Тип оборудования'][0]
fail_point = df['Точка отказа'][0]

In [11]:
body

'Здравствуйте.\r\n\r\n                Сломался блок питания (отвалился разъём type-c) у ноутбука СИЛА НК2-1404 серийный номер C222090381.\r\n\r\nЭксплуатируется в Волгограде.'

In [ ]:
template = """Представь, что ты помощник специалиста пользовательской поддержки компании по производству и обслуживанию технического оборудования.

Тебе поступило письмо по электронной почте от клиента.

Заголовок письма: {title}.

Текст письма: {body}.

Проанализируй, достаточно ли информации из письма для того, чтобы диспетчер понял, что проблема с оборудованием {type_of_equipment}, а также его частью {fail_point}.

Ответ напиши в виде json-файла, в котором будет поле flag и поле answer.

Если ты считаешь, что информации достаточно для того, чтобы понять, что проблема с оборудованием {type_of_equipment}, а также его частью {fail_point}, тогда в поле flag напиши True, а поле answer сделай пустым.

Если ты считаешь, что информации не достаточно для того, чтобы понять, что проблема с оборудованием {type_of_equipment}, а также его частью {fail_point}, тогда в поле flag напиши False, а в поле answer напиши с точки зрения оператора поддержки, что необходимо добавить в письме, чтобы оно стало понятным.
Отвечай так, как если бы ты лично общалась с клиентом. Будь вежливой, поздоровайся с ним, а потом укажи кратко по пунктам, чего не хватает.

Следуй инструкциям, отвечай кратко, но по делу. В ответе кроме json-файла ничего писать не нужно.

Структура ответа:

{{
  'flag': ...,
  'answer': ...,
}}

"""

prompt = ChatPromptTemplate.from_template(template)

llm = OllamaLLM(
    model='gemma2',
    # model='llama3.1:8b',
    base_url='http://26.251.162.207:11434',
    temperature=0,
    format='json',
    num_ctx=8192,
    num_predict=128,
)

chain_ = prompt | llm #| (lambda x: x.strip(' \n\r.'))


chain_.invoke({
        'title': title,
        'body': body,
        'type_of_equipment': type_of_equipment,
        'fail_point': fail_point,
    })

'{\n  "flag": true,\n  "answer": ""\n} \n'

In [ ]:
def check_prompt(df, index):
    title = df.iloc[index]['Тема']
    body = df.iloc[index]['Описание']


    template = """Представь, что ты помощник специалиста пользовательской поддержки компании по производству и обслуживанию технического оборудования.

    Тебе поступило письмо по электронной почте от клиента.

    Заголовок письма: {title}.

    Текст письма: {body}.

    Проанализируй, достаточно ли информации из письма для того, чтобы узнать название модели оборудования. Пример названия модели оборудования: НК2-1404 или CX-1015-12.
    Оно также может быть окружено символами //.

    Ответ напиши в виде json-файла, в котором будет поле flag и поле answer.

    Если ты считаешь, что информации достаточно для того, чтобы понять название модели оборудования, тогда в поле flag напиши True, а поле answer сделай пустым.

    Если ты считаешь, что информации не достаточно для того, чтобы понять название модели оборудования, тогда в поле flag напиши False, а в поле answer напиши с точки зрения оператора поддержки, что необходимо добавить в письме, чтобы оно стало понятным.
    Отвечай так, как если бы ты лично общалась с клиентом. Будь вежливой, поздоровайся с ним, а потом укажи кратко по пунктам, чего не хватает. Не приводи примеры названий.

    Следуй инструкциям, отвечай кратко, но по делу. Думай дважды. Не сильно концетрируйся на контексте всего письма, только на название моделей. В ответе кроме json-файла ничего писать не нужно.

    Структура ответа:

    {{
    'flag': ...,
    'answer': ...,
    }}

    """

    prompt = ChatPromptTemplate.from_template(template)

    llm = OllamaLLM(
        model='gemma2',
        #model='llama3.1:8b',
        base_url='http://26.251.162.207:11434',
        #base_url='http://26.194.16.61:11434',
        temperature=0,
        format='json',
        num_ctx=8192,
        num_predict=128,
    )

    chain_ = prompt | llm #| (lambda x: x.strip(' \n\r.'))


    return chain_.invoke({
            'title': title,
            'body': body
        }), title, body

In [ ]:
HK-1404

In [65]:
"HK2" == "HK2"

True

In [ ]:
print(df.loc[df['Тип оборудования'] == 'СХД'][['Тема', 'Описание']].values.sum())

dd503.region..ru - Disk 3.2Неисправность диска Enclosure=6:Disk=32 dd202Просьба открыть заявку по неисправному диску на dd203 - Enclosure=5:Disk=31Просьба открыть заявку по неисправному диску:  dd401..ru - Disk 3.23 SN: ZC1BYZ1XСИЛА CX-1015-12 // HDD fault //  s/n . Вышел из строя диск, пришедший на замену по гарантииСИЛА СХ2-4115-12; СИЛА CX2-4016-25; СИЛА СХ2-4066-36  //  обновить ПО до последней версии // ,СХД CX4-1127-00 // Неисправный дисквылетели 9 дисков и перезагрузилась резервная нода dd503открыть кейс по неисправному диску на dd:  dd203..ru  Enclosure=3:Disk=49открыть кейс по неисправному диску на dd:  dd204.region..ru  Enclosure=2:Disk=12по неисправной плашке памяти на dd203 - Enclosure=1:DIMM=MR2 DIMM A0Добрый день.
Прошу открыть заявку на замену диска.
dd503.region..ru - Disk 3.2
@dd503-p1(active:1)# system show serial
Serial number: CKM01212505744
@dd503-p1(active:1)# system show model
Model number: DD9900 E
@dd503-p1(active:1)# alerts show current
Id      Post Time      

In [149]:
df_test = df.loc[df['Тип оборудования'] == 'СХД']

In [172]:
answer, title, body = check_prompt(df, 22)
print(answer)
print(title)
print(body)

{
"flag": false,
"answer": "Здравствуйте! Чтобы мы могли разобраться с ситуацией и помочь вам, пожалуйста, уточните модель вашего сервера.  Также будет полезно узнать версию прошивки." 
} 

Аварийная перезагрузка сервера,  UMA operation error
Аварийно перезагрузился сервер, в логе BMC появилось совпадающее по времени событие:
20	 Critical	2024-05-31 17:38:27 (UTC+3)
Intel ME has encountered UMA operation error. Details: UMA Read/Write timeout. Timeout occurred during copying data from/to UMA.

Просьба продиагностировать, возможно ли использовать данный сервер в продуктивном контуре
